In [14]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [21]:
training_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor()
)

test_data = datasets.FashionMNIST(
    root = "data",
    train = False,
    download = True,
    transform = ToTensor()
)

In [22]:
batch_size = 64
train_dataloader = DataLoader(training_data, batch_size = batch_size)
test_dataloader = DataLoader(test_data, batch_size = batch_size)

In [23]:
device = (
    "cuda" 
    if torch.cuda.is_available() 
    else "mps"
    if torch.backends.mps.is_available() 
    else "cpu"
)

print(device)

mps


In [26]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    

model = NeuralNetwork().to(device)
print(model)


NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [29]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [30]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
    

In [31]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [35]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)

print('Done!')

Epoch 1
-------------------------------
loss: 1.021442 [   64/60000]
loss: 1.051626 [ 6464/60000]
loss: 0.845584 [12864/60000]
loss: 1.004675 [19264/60000]
loss: 0.879796 [25664/60000]
loss: 0.899094 [32064/60000]
loss: 0.967097 [38464/60000]
loss: 0.901020 [44864/60000]
loss: 0.940126 [51264/60000]
loss: 0.891060 [57664/60000]
Test Error: 
 Accuracy: 67.6%, Avg loss: 0.898365 

Epoch 2
-------------------------------
loss: 0.937184 [   64/60000]
loss: 0.987608 [ 6464/60000]
loss: 0.766817 [12864/60000]
loss: 0.942644 [19264/60000]
loss: 0.825202 [25664/60000]
loss: 0.831954 [32064/60000]
loss: 0.915695 [38464/60000]
loss: 0.852573 [44864/60000]
loss: 0.885224 [51264/60000]
loss: 0.844682 [57664/60000]
Test Error: 
 Accuracy: 68.7%, Avg loss: 0.848306 

Epoch 3
-------------------------------
loss: 0.872297 [   64/60000]
loss: 0.938874 [ 6464/60000]
loss: 0.707365 [12864/60000]
loss: 0.895919 [19264/60000]
loss: 0.786120 [25664/60000]
loss: 0.781376 [32064/60000]
loss: 0.876317 [38464/

Save Model

In [36]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [55]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
print(type(x))
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    print(pred, pred.shape, pred.ndim)
    print(type(pred), type(pred[0]))
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: {predicted}, Actual: {actual}')
    

<class 'torch.Tensor'>
tensor([[-4.0878, -4.6553, -3.0625, -3.0662, -1.9257,  5.7704, -2.9318,  5.5002,
          2.4612,  6.3232]], device='mps:0') torch.Size([1, 10]) 2
<class 'torch.Tensor'> <class 'torch.Tensor'>
Predicted: Ankle boot, Actual: Ankle boot
